Tak Kit Yeung

Scrape CCNY calendar:

In [1]:
# import libraries
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import timedelta

In [2]:
# send HTTP requests
url = 'https://www.ccny.cuny.edu/registrar/fall'
response = requests.get(url)
# extract data from HTML
soup = BeautifulSoup(response.text)
td = soup.find_all('td')

Data of interest in html:

In [3]:
print(td)

[<td style="width:305px">
<p><strong>August 01</strong></p>
</td>, <td style="width:327px">
<p><strong>Sunday</strong></p>
</td>, <td style="width:617px">
<p><strong>Application for degree for January and February 2022 begins</strong></p>
</td>, <td style="width:305px">
<p>August 18</p>
</td>, <td style="width:327px">
<p>Wednesday</p>
</td>, <td style="width:617px">
<p>Last day to apply for Study Abroad</p>
</td>, <td style="width:305px">
<p><strong>August 24</strong></p>
</td>, <td style="width:327px">
<p><strong>Tuesday</strong></p>
</td>, <td style="width:617px">
<p><strong>Last day of Registration;<br/>
			Last day to file ePermit for the Fall 2021;<br/>
			Last day to drop classes for 100% tuition refund;</strong></p>
</td>, <td style="width:305px">
<p>August 25</p>
</td>, <td style="width:327px">
<p>Wednesday</p>
</td>, <td style="width:617px">
<p>Start of Fall Term;<br/>
			Classes begin;<br/>
			Initial Registration Appeals begin;</p>
</td>, <td style="width:305px">
<p><strong>

Create dataframe with string as index column:

In [4]:
# transform data
t = []
r = []
for i in range(len(td)):
    # remove redundant whitespace
    d = td[i].get_text().replace('\n','').replace('\t',' ')
    r.append(d)
    # append rows
    if i % 3 == 2:
        t.append(r)
        r = []

In [5]:
# create dataframe
df = pd.DataFrame(t, columns = ['date', 'dow', 'text'])
df.set_index('date', inplace = True)
# display dataframe
pd.set_option('display.max_colwidth', 0)
display(df)

,dow,text
date,,
August 01,Sunday,Application for degree for January and February 2022 begins
August 18,Wednesday,Last day to apply for Study Abroad
August 24,Tuesday,Last day of Registration; Last day to file ePermit for the Fall 2021; Last day to drop classes for 100% tuition refund;
August 25,Wednesday,Start of Fall Term; Classes begin; Initial Registration Appeals begin;
August 25 - 31,Wednesday - Tuesday,Change of program period; late fees apply
August 26,Thursday,Last day for Independent Study
August 28,Saturday,First day of Saturday Classes
August 31,Tuesday,Last day to add a class to an existing enrollment; Last day for 75% tuition refund; Financial Aid Certification Enrollment Status date; Last day to apply for Audit option; Last day to drop without the grade of 'WD'; Initial Registration Appeals end;
September 01,Wednesday,Verification of Enrollment rosters available to faculty; Course Withdrawal drop period begins (A grade of 'WD' is assigned to students who officially drop a class);


Create dataframe with date as index column:

In [6]:
# transform data
table = []
row = []
dayName = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
for i in range(len(td)):
    # remove redundant whitespace
    data = td[i].get_text().replace('\n','').replace('\t',' ')
    # convert data to date and weekday
    if i % 3 == 0:
        dates = data.split(' - ')
        # add year to date if missing
        for i in range(len(dates)):
            if '2022' not in dates[i]:
                dates[i] += ' 2021'
        data = pd.to_datetime(dates[0]).date()
        row.append(data)
        row.append(dayName[data.weekday()])
    # append rows
    if i % 3 == 2:
        row.append(data)
        table.append(row)
        # handle multi-day event
        if len(dates) > 1:
            # add month to date if missing
            month = ''
            for c in dates[0]:
                if c == ' ':
                    break
                else:
                    month += c
            if dates[1][0].isdigit():
                dateEnd = pd.to_datetime(month + ' ' + dates[1]).date()
            else:
                dateEnd = pd.to_datetime(dates[1]).date()
            # calculate number of days
            dateDiff = (dateEnd - row[0]).days
            for j in range(1, dateDiff + 1):
                row2 = []
                dateNow = row[0] + timedelta(days = j)
                row2.append(dateNow)
                row2.append(dayName[dateNow.weekday()])
                row2.append(row[2])
                table.append(row2)
        row = []

In [7]:
# create dataframe
df2 = pd.DataFrame(table, columns = ['date', 'dow', 'text'])
df2.set_index('date', inplace = True)
# display dataframe
display(df2)

,dow,text
date,,
2021-08-01,Sunday,Application for degree for January and February 2022 begins
2021-08-18,Wednesday,Last day to apply for Study Abroad
2021-08-24,Tuesday,Last day of Registration; Last day to file ePermit for the Fall 2021; Last day to drop classes for 100% tuition refund;
2021-08-25,Wednesday,Start of Fall Term; Classes begin; Initial Registration Appeals begin;
2021-08-25,Wednesday,Change of program period; late fees apply
2021-08-26,Thursday,Change of program period; late fees apply
2021-08-27,Friday,Change of program period; late fees apply
2021-08-28,Saturday,Change of program period; late fees apply
2021-08-29,Sunday,Change of program period; late fees apply
